In [1]:
import sympy as sm
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import timeit

In [2]:
# r, K, aC, eC, hC, mC, aP, eP, hP, mP, q = sm.symbols('r, K, a_C, e_C, h_C, m_C, a_P, e_P, h_P, m_P,q', positive=True)
# R, C, P = sm.symbols('R, C, P', negative=False)


# Rdt = r*R*(1 - (R*q)) - (aC*R*C)/(1 + (aC*hC*R))
# Cdt = (eC*aC*R*C)/(1 + (aC*hC*R)) - (aP*C*P)/(1+ (aP*hP*C)) - mC*C
# Pdt = (eP*aP*C*P)/(1+ (aP*hP*C)) - mP*P

# sm.Matrix([Rdt, Cdt, Pdt])

In [3]:
# Ms = sm.Matrix([Rdt, Cdt, Pdt])
# Msv = Ms.jacobian([R, C, P])

# m00 = Msv[0,0].simplify()
# m01 = Msv[0,1].simplify()
# m02 = 0

# m10 = Msv[1,0].simplify()
# m11 = Msv[1,1].simplify()
# m12 = Msv[1,2].simplify()

# m20 = 0
# m21 = Msv[2,1].simplify()
# m22 = Msv[2,2]

# Jss = sm.Matrix([[m00, m01, m02],[m10, m11, m12],[m20, m21, m22]])
# Jss

In [4]:
pars = pd.read_csv("C:/SDMS/trophic_temp/param_hycub_H2.csv")
par1 = np.array(pars)
np.shape(par1)

(10, 1000)

In [8]:
def Lyapunov_Jac_RCP (t, u, r1, q1, aC1, eC1, hC1, mC1, aP1, eP1, mP1, hP1):
    
    du = np.zeros([12, 1])
    
    #variables
    R = u[0]
    C = u[1]
    P = u[2]
    
    #State variables
    du[0] = r1*R*(1 - (R*q1)) - (aC1*R*C)/(1 + (aC1*hC1*R))
    du[1] = (eC1*aC1*R*C)/(1 + (aC1*hC1*R)) - (aP1*C*P)/(1+ (aP1*hP1*C)) - mC1*C
    du[2] = (eP1*aP1*C*P)/(1+ (aP1*hP1*C)) - mP1*P
    
    ##Jacobian calculation
    Jab11 = -C*aC1/(R*aC1*hC1 + 1)**2 - 2*R*q1*r1 + r1
    Jab12 = -R*aC1/(R*aC1*hC1 + 1)
    Jab13 = 0

    Jab21 = C*aC1*eC1/(R*aC1*hC1 + 1)**2
    Jab22 = C*P*aP1**2*hP1/(C*aP1*hP1 + 1)**2 - P*aP1/(C*aP1*hP1 + 1) + R*aC1*eC1/(R*aC1*hC1 + 1) - mC1
    Jab23 = -C*aP1/(C*aP1*hP1 + 1)

    Jab31 = 0
    Jab32 = P*aP1*eP1/(C*aP1*hP1 + 1)**2
    Jab33 = C*aP1*eP1/(C*aP1*hP1 + 1) - mP1
    
    ##Jacobian in matrix form
    Jx = np.array([[Jab11, Jab12, Jab13],[Jab21, Jab22, Jab23],[Jab31, Jab32, Jab33]])
    Dx0 = u[np.arange(3, 3+9)].reshape(3,3)

    dDx0 = (np.matmul(Jx,Dx0))
    dDxv0 = dDx0.reshape(9, 1)
    du[np.arange(3, 3+9)] = dDxv0
    
    du = du.reshape(12,)
    return(du)
    

In [ ]:
i=27
r0 = par1[0][i]; q0 =  par1[1][i]
aC0 = par1[2][i]; eC0 = par1[3][i]; mC0 = par1[4][i]; hC0 = par1[5][i]
aP0 = par1[6][i]; eP0 = par1[7][i]; mP0 = par1[8][i]; hP0 = par1[9][i]

u00 = np.array([5000, 280, 58])
p = (r0, q0, aC0, eC0, hC0, mC0, aP0, eP0, mP0, hP0)
   
tend = 100000
t_span = (0, tend)
tint = np.linspace(0, tend, tend*2)

u0 = np.concatenate((u00, np.eye(3).reshape(9)), axis=0)
sol1 = solve_ivp (Lyapunov_Jac_RCP, t_span, u0, args = p, t_eval = tint, method= 'RK45', rtol=1e-06, atol=1e-06)

print(np.shape(sol1.y))
print(np.shape(sol1.t))

In [ ]:
Dxv = (sol1.y[np.arange(3,3+9),:]).T
Ts = np.shape(Dxv)[0]
print(np.shape(Dxv))
Dxv

In [ ]:
##Lyapunov exponents
Dxv = (sol1.y[np.arange(3,3+9),:]).T
Ts = np.shape(Dxv)[0]

lmbda = np.empty([Ts,3])*np.nan; Nexpx0 = np.empty(Ts)*np.nan;

for i in np.arange(Ts):
    Dxi = Dxv[i,:].reshape(3,3)
    Mi = np.matmul(Dxi.T, Dxi)
    
    
    Mi1 = np.count_nonzero(Mi == np.inf)
    Mi2 = np.count_nonzero(Mi == np.nan)
    Mii = Mi1 + Mi2
    
    if Mii == 0:
        Uxi,Lxi,Vxi = np.linalg.svd(Mi, full_matrices=True)
        Ldxi = np.diag(Lxi);

        ti = tint[i];
        lambdai = np.log(Lxi)/(2*ti)
        Nexpx0i = len(lambdai[lambdai > 0])

        lmbda[i,:] = lambdai
        Nexpx0[i] = Nexpx0i

In [ ]:
lmbda[-10:]

In [ ]:
Nexpx0[-100:]